In [205]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

## Data Cleaning and Processing

#### Housing Inventory Counts by State

In [206]:
years = list(map(str, range(2014, 2023)))

# Read 2012-2022 sheets of an Excel file into a dictionary of DataFrames
dict_of_dfs = {year: pd.read_excel('2007-2022-HIC-Counts-by-State.xlsx', sheet_name=year, skiprows=1) for year in years}

In [207]:
# Find the common set of columns
common_columns = set.intersection(*[set(df.columns) for df in dict_of_dfs.values()])

In [208]:
# Realign data and add "Year" column
all_data = pd.DataFrame()
for year, df in dict_of_dfs.items():
    missing_cols = common_columns - set(df.columns)
    extra_cols = set(df.columns) - common_columns
    df.drop(columns=extra_cols, errors='ignore', inplace=True)
    for col in missing_cols:
        df[col] = np.nan
    df['Year'] = year
    all_data = pd.concat([all_data, df], ignore_index=True)

In [209]:
# Concatenate all of the DataFrames in the dictionary into a single DataFrame
HIC_Counts = pd.concat(dict_of_dfs.values(), ignore_index=True)

In [210]:
# Remove US Territories and Total rows for each year
HIC_Counts = HIC_Counts[~HIC_Counts['State'].isin(['MP', 'GU', 'VI', 'PR', 'Total'])]

In [211]:
row_count = HIC_Counts.shape[0]
print("Number of rows:", row_count)

Number of rows: 459


In [212]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [213]:
HIC_Counts.dtypes

State                                                         object
Total Year-Round Beds (ES, TH, SH)                             int64
Total Non-DV Year-Round Beds (ES, TH, SH)                      int64
Total HMIS Year-Round Beds (ES, TH, SH)                        int64
HMIS Participation Rate for Year-Round Beds (ES, TH, SH)     float64
Total Year-Round Beds (ES)                                     int64
Total Year-Round Beds (TH)                                     int64
Total Year-Round Beds (SH)                                     int64
Total Units for Households with Children (ES, TH, SH)          int64
Total Beds for Households with Children (ES, TH, SH)           int64
Total Beds for Households without Children (ES, TH, SH)        int64
Total Beds for Households with only Children (ES, TH, SH)      int64
Total Year-Round Beds (ES).1                                   int64
Total Non-DV Year-Round Beds (ES)                              int64
Total HMIS Year-Round Beds (ES)   

In [214]:
participation_columns = ['HMIS Participation Rate for Year-Round Beds (ES, TH, SH)',
                         'HMIS Participation Rate for Year-Round Beds (ES)',
                         'HMIS Participation Rate for Year-Round Beds (TH)',
                         'HMIS Participation Rate for Year-Round Beds (SH)',
                         'HMIS Participation Rate for Year-Round Beds (RRH)',
                         'HMIS Participation Rate for Year-Round Beds (PSH)',
                         'HMIS Participation Rate for Year-Round Beds (OPH)']

# Replace '.' with NaN
HIC_Counts[participation_columns] = HIC_Counts[participation_columns].replace('.', np.nan)

# Convert to float
HIC_Counts[participation_columns] = HIC_Counts[participation_columns].astype(float)

In [215]:
missing_values = HIC_Counts.isnull().sum()
missing_values

State                                                          0
Total Year-Round Beds (ES, TH, SH)                             0
Total Non-DV Year-Round Beds (ES, TH, SH)                      0
Total HMIS Year-Round Beds (ES, TH, SH)                        0
HMIS Participation Rate for Year-Round Beds (ES, TH, SH)       0
Total Year-Round Beds (ES)                                     0
Total Year-Round Beds (TH)                                     0
Total Year-Round Beds (SH)                                     0
Total Units for Households with Children (ES, TH, SH)          0
Total Beds for Households with Children (ES, TH, SH)           0
Total Beds for Households without Children (ES, TH, SH)        0
Total Beds for Households with only Children (ES, TH, SH)      0
Total Year-Round Beds (ES).1                                   0
Total Non-DV Year-Round Beds (ES)                              0
Total HMIS Year-Round Beds (ES)                                0
HMIS Participation Rate f

In [216]:
HIC_Counts['HMIS Participation Rate for Year-Round Beds (OPH)'] = HIC_Counts['HMIS Participation Rate for Year-Round Beds (OPH)'].fillna('0%')
HIC_Counts['HMIS Participation Rate for Year-Round Beds (SH)'] = HIC_Counts['HMIS Participation Rate for Year-Round Beds (SH)'].fillna('0%')
HIC_Counts['HMIS Participation Rate for Year-Round Beds (RRH)'] = HIC_Counts['HMIS Participation Rate for Year-Round Beds (RRH)'].fillna('0%')

In [217]:
missing_values = HIC_Counts.isnull().sum()
missing_values

State                                                        0
Total Year-Round Beds (ES, TH, SH)                           0
Total Non-DV Year-Round Beds (ES, TH, SH)                    0
Total HMIS Year-Round Beds (ES, TH, SH)                      0
HMIS Participation Rate for Year-Round Beds (ES, TH, SH)     0
Total Year-Round Beds (ES)                                   0
Total Year-Round Beds (TH)                                   0
Total Year-Round Beds (SH)                                   0
Total Units for Households with Children (ES, TH, SH)        0
Total Beds for Households with Children (ES, TH, SH)         0
Total Beds for Households without Children (ES, TH, SH)      0
Total Beds for Households with only Children (ES, TH, SH)    0
Total Year-Round Beds (ES).1                                 0
Total Non-DV Year-Round Beds (ES)                            0
Total HMIS Year-Round Beds (ES)                              0
HMIS Participation Rate for Year-Round Beds (ES)       

In [218]:
southeast = {'AL', 'AR', 'FL', 'GA', 'KY', 'LA', 'MS', 'NC', 'SC', 'TN', 'VA', 'DC', 'WV'}
northeast = {'CT', 'DE', 'ME', 'MD', 'MA', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT', 'DC'}
west = {'AK', 'CA', 'CO', 'HI', 'ID', 'MT', 'NV', 'OR', 'UT', 'WA', 'WY'}
midwest = {'IL', 'IN', 'IA', 'KS', 'MI', 'MN', 'MO', 'NE', 'ND', 'OH', 'SD', 'WI'}
southwest = {'AZ', 'NM', 'OK', 'TX'}

state_to_region = {state: 'Southeast' for state in southeast}
state_to_region.update({state: 'Northeast' for state in northeast})
state_to_region.update({state: 'West' for state in west})
state_to_region.update({state: 'Midwest' for state in midwest})
state_to_region.update({state: 'Southwest' for state in southwest})

In [219]:
# Add Region column 
HIC_Counts['Region'] = HIC_Counts['State'].map(state_to_region)

#### Census Data 

In [220]:
census_data_pre2020 = pd.read_csv('Census2010_to_2019.csv', skiprows=3)

In [221]:
New_Labels = ['Geographic Area', 'Census', 'Estimates Base', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
census_data_pre2020.columns = New_Labels

In [222]:
census_data_pre2020.head()

,Geographic Area,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,"308,745,538","308,758,105","309,321,666","311,556,874","313,830,990","315,993,715","318,301,008","320,635,163","322,941,311","324,985,539","326,687,501","328,239,523"
1,Northeast,"55,317,240","55,318,443","55,380,134","55,604,223","55,775,216","55,901,806","56,006,011","56,034,684","56,042,330","56,059,240","56,046,620","55,982,803"
2,Midwest,"66,927,001","66,929,725","66,974,416","67,157,800","67,336,743","67,560,379","67,745,167","67,860,583","67,987,540","68,126,781","68,236,628","68,329,004"
3,South,"114,555,744","114,563,030","114,866,680","116,006,522","117,241,208","118,364,400","119,624,037","120,997,341","122,351,760","123,542,189","124,569,433","125,580,448"
4,West,"71,945,553","71,946,907","72,100,436","72,788,329","73,477,823","74,167,130","74,925,793","75,742,555","76,559,681","77,257,329","77,834,820","78,347,268"


In [223]:
census_data_post2020 = pd.read_csv('Census2020_to_2022.csv', skiprows=3)

In [224]:
New_Labels = ['Geographic Area', 'Estimates Base', '2020', '2021', '2022']
census_data_post2020.columns = New_Labels

In [225]:
census_data_post2020.head()

,Geographic Area,Estimates Base,2020,2021,2022
0,United States,"331,449,520","331,511,512","332,031,554","333,287,557"
1,Northeast,"57,609,156","57,448,898","57,259,257","57,040,406"
2,Midwest,"68,985,537","68,961,043","68,836,505","68,787,595"
3,South,"126,266,262","126,450,613","127,346,029","128,716,192"
4,West,"78,588,565","78,650,958","78,589,763","78,743,364"


In [226]:
census_data = census_data_pre2020.merge(census_data_post2020, on='Geographic Area')
census_data.head(10)

,Geographic Area,Census,Estimates Base_x,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Estimates Base_y,2020,2021,2022
0,United States,"308,745,538","308,758,105","309,321,666","311,556,874","313,830,990","315,993,715","318,301,008","320,635,163","322,941,311","324,985,539","326,687,501","328,239,523","331,449,520","331,511,512","332,031,554","333,287,557"
1,Northeast,"55,317,240","55,318,443","55,380,134","55,604,223","55,775,216","55,901,806","56,006,011","56,034,684","56,042,330","56,059,240","56,046,620","55,982,803","57,609,156","57,448,898","57,259,257","57,040,406"
2,Midwest,"66,927,001","66,929,725","66,974,416","67,157,800","67,336,743","67,560,379","67,745,167","67,860,583","67,987,540","68,126,781","68,236,628","68,329,004","68,985,537","68,961,043","68,836,505","68,787,595"
3,South,"114,555,744","114,563,030","114,866,680","116,006,522","117,241,208","118,364,400","119,624,037","120,997,341","122,351,760","123,542,189","124,569,433","125,580,448","126,266,262","126,450,613","127,346,029","128,716,192"
4,West,"71,945,553","71,946,907","72,100,436","72,788,329","73,477,823","74,167,130","74,925,793","75,742,555","76,559,681","77,257,329","77,834,820","78,347,268","78,588,565","78,650,958","78,589,763","78,743,364"
5,.Alabama,"4,779,736","4,780,125","4,785,437","4,799,069","4,815,588","4,830,081","4,841,799","4,852,347","4,863,525","4,874,486","4,887,681","4,903,185","5,024,356","5,031,362","5,049,846","5,074,296"
6,.Alaska,"710,231","710,249","713,910","722,128","730,443","737,068","736,283","737,498","741,456","739,700","735,139","731,545","733,378","732,923","734,182","733,583"
7,.Arizona,"6,392,017","6,392,288","6,407,172","6,472,643","6,554,978","6,632,764","6,730,413","6,829,676","6,941,072","7,044,008","7,158,024","7,278,717","7,151,507","7,179,943","7,264,877","7,359,197"
8,.Arkansas,"2,915,918","2,916,031","2,921,964","2,940,667","2,952,164","2,959,400","2,967,392","2,978,048","2,989,918","3,001,345","3,009,733","3,017,804","3,011,555","3,014,195","3,028,122","3,045,637"
9,.California,"37,253,956","37,254,519","37,319,502","37,638,369","37,948,800","38,260,787","38,596,972","38,918,045","39,167,117","39,358,497","39,461,588","39,512,223","39,538,245","39,501,653","39,142,991","39,029,342"


In [227]:
state_abbr = {
    '.Alabama': 'AL',
    '.Alaska': 'AK',
    '.Arizona': 'AZ',
    '.Arkansas': 'AR',
    '.California': 'CA',
    '.Colorado': 'CO',
    '.Connecticut': 'CT',
    '.Delaware': 'DE',
    '.Florida': 'FL',
    '.Georgia': 'GA',
    '.Hawaii': 'HI',
    '.Idaho': 'ID',
    '.Illinois': 'IL',
    '.Indiana': 'IN',
    '.Iowa': 'IA',
    '.Kansas': 'KS',
    '.Kentucky': 'KY',
    '.Louisiana': 'LA',
    '.Maine': 'ME',
    '.Maryland': 'MD',
    '.Massachusetts': 'MA',
    '.Michigan': 'MI',
    '.Minnesota': 'MN',
    '.Mississippi': 'MS',
    '.Missouri': 'MO',
    '.Montana': 'MT',
    '.Nebraska': 'NE',
    '.Nevada': 'NV',
    '.New Hampshire': 'NH',
    '.New Jersey': 'NJ',
    '.New Mexico': 'NM',
    '.New York': 'NY',
    '.North Carolina': 'NC',
    '.North Dakota': 'ND',
    '.Ohio': 'OH',
    '.Oklahoma': 'OK',
    '.Oregon': 'OR',
    '.Pennsylvania': 'PA',
    '.Rhode Island': 'RI',
    '.South Carolina': 'SC',
    '.South Dakota': 'SD',
    '.Tennessee': 'TN',
    '.Texas': 'TX',
    '.Utah': 'UT',
    '.Vermont': 'VT',
    '.Virginia': 'VA',
    '.Washington': 'WA',
    '.West Virginia': 'WV',
    '.Wisconsin': 'WI',
    '.Wyoming': 'WY',
    '.District of Columbia': 'DC'
}

census_data['Geographic Area'] = census_data['Geographic Area'].map(state_abbr)

In [228]:
census_data.head(10)

,Geographic Area,Census,Estimates Base_x,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Estimates Base_y,2020,2021,2022
0,NaN,"308,745,538","308,758,105","309,321,666","311,556,874","313,830,990","315,993,715","318,301,008","320,635,163","322,941,311","324,985,539","326,687,501","328,239,523","331,449,520","331,511,512","332,031,554","333,287,557"
1,NaN,"55,317,240","55,318,443","55,380,134","55,604,223","55,775,216","55,901,806","56,006,011","56,034,684","56,042,330","56,059,240","56,046,620","55,982,803","57,609,156","57,448,898","57,259,257","57,040,406"
2,NaN,"66,927,001","66,929,725","66,974,416","67,157,800","67,336,743","67,560,379","67,745,167","67,860,583","67,987,540","68,126,781","68,236,628","68,329,004","68,985,537","68,961,043","68,836,505","68,787,595"
3,NaN,"114,555,744","114,563,030","114,866,680","116,006,522","117,241,208","118,364,400","119,624,037","120,997,341","122,351,760","123,542,189","124,569,433","125,580,448","126,266,262","126,450,613","127,346,029","128,716,192"
4,NaN,"71,945,553","71,946,907","72,100,436","72,788,329","73,477,823","74,167,130","74,925,793","75,742,555","76,559,681","77,257,329","77,834,820","78,347,268","78,588,565","78,650,958","78,589,763","78,743,364"
5,AL,"4,779,736","4,780,125","4,785,437","4,799,069","4,815,588","4,830,081","4,841,799","4,852,347","4,863,525","4,874,486","4,887,681","4,903,185","5,024,356","5,031,362","5,049,846","5,074,296"
6,AK,"710,231","710,249","713,910","722,128","730,443","737,068","736,283","737,498","741,456","739,700","735,139","731,545","733,378","732,923","734,182","733,583"
7,AZ,"6,392,017","6,392,288","6,407,172","6,472,643","6,554,978","6,632,764","6,730,413","6,829,676","6,941,072","7,044,008","7,158,024","7,278,717","7,151,507","7,179,943","7,264,877","7,359,197"
8,AR,"2,915,918","2,916,031","2,921,964","2,940,667","2,952,164","2,959,400","2,967,392","2,978,048","2,989,918","3,001,345","3,009,733","3,017,804","3,011,555","3,014,195","3,028,122","3,045,637"
9,CA,"37,253,956","37,254,519","37,319,502","37,638,369","37,948,800","38,260,787","38,596,972","38,918,045","39,167,117","39,358,497","39,461,588","39,512,223","39,538,245","39,501,653","39,142,991","39,029,342"


In [229]:
census_data = census_data.dropna(subset=['Geographic Area'])
census_data.head()

,Geographic Area,Census,Estimates Base_x,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Estimates Base_y,2020,2021,2022
5,AL,"4,779,736","4,780,125","4,785,437","4,799,069","4,815,588","4,830,081","4,841,799","4,852,347","4,863,525","4,874,486","4,887,681","4,903,185","5,024,356","5,031,362","5,049,846","5,074,296"
6,AK,"710,231","710,249","713,910","722,128","730,443","737,068","736,283","737,498","741,456","739,700","735,139","731,545","733,378","732,923","734,182","733,583"
7,AZ,"6,392,017","6,392,288","6,407,172","6,472,643","6,554,978","6,632,764","6,730,413","6,829,676","6,941,072","7,044,008","7,158,024","7,278,717","7,151,507","7,179,943","7,264,877","7,359,197"
8,AR,"2,915,918","2,916,031","2,921,964","2,940,667","2,952,164","2,959,400","2,967,392","2,978,048","2,989,918","3,001,345","3,009,733","3,017,804","3,011,555","3,014,195","3,028,122","3,045,637"
9,CA,"37,253,956","37,254,519","37,319,502","37,638,369","37,948,800","38,260,787","38,596,972","38,918,045","39,167,117","39,358,497","39,461,588","39,512,223","39,538,245","39,501,653","39,142,991","39,029,342"


#### Adding Census Data to Housing Inventory Counts by State

In [230]:
HIC_Counts['Population'] = np.nan

In [231]:
for index, row in HIC_Counts.iterrows():
    state = row['State']
    year = row['Year']
    population = census_data.loc[census_data['Geographic Area'] == state, str(year)].values[0]
    HIC_Counts.at[index, 'Population'] = population

In [232]:
HIC_Counts.head()

,State,"Total Year-Round Beds (ES, TH, SH)","Total Non-DV Year-Round Beds (ES, TH, SH)","Total HMIS Year-Round Beds (ES, TH, SH)","HMIS Participation Rate for Year-Round Beds (ES, TH, SH)",Total Year-Round Beds (ES),Total Year-Round Beds (TH),Total Year-Round Beds (SH),"Total Units for Households with Children (ES, TH, SH)","Total Beds for Households with Children (ES, TH, SH)","Total Beds for Households without Children (ES, TH, SH)","Total Beds for Households with only Children (ES, TH, SH)",Total Year-Round Beds (ES).1,Total Non-DV Year-Round Beds (ES),Total HMIS Year-Round Beds (ES),HMIS Participation Rate for Year-Round Beds (ES),Total Seasonal Beds (ES),Total Overflow Beds (ES),Total Units for Households with Children (ES),Total Beds for Households with Children (ES),Total Beds for Households without Children (ES),Total Beds for Households with only Children (ES),Total Year-Round Beds (TH).1,Total Non-DV Year-Round Beds (TH),Total HMIS Year-Round Beds (TH),HMIS Participation Rate for Year-Round Beds (TH),Total Units for Households with Children (TH),Total Beds for Households with Children (TH),Total Beds for Households without Children (TH),Total Beds for Households with only Children (TH),Total Year-Round Beds (SH).1,Total Non-DV Year-Round Beds (SH),Total HMIS Year-Round Beds (SH),HMIS Participation Rate for Year-Round Beds (SH),Total Units for Households with Children (SH),Total Beds for Households with Children (SH),Total Beds for Households without Children (SH),Total Beds for Households with only Children (SH),Total Year-Round Beds (RRH),Total Non-DV Year-Round Beds (RRH),Total HMIS Year-Round Beds (RRH),HMIS Participation Rate for Year-Round Beds (RRH),Total Units for Households with Children (RRH),Total Beds for Households with Children (RRH),Total Beds for Households without Children (RRH),Total Beds for Households with only Children (RRH),Total Year-Round Beds (PSH),Total Non-DV Year-Round Beds (PSH),Total HMIS Year-Round Beds (PSH),HMIS Participation Rate for Year-Round Beds (PSH),Total Units for Households with Children (PSH),Total Beds for Households with Children (PSH),Total Beds for Households without Children (PSH),Total Beds for Households with only Children (PSH),Total Year-Round Beds (OPH),Total Non-DV Year-Round Beds (OPH),Total HMIS Year-Round Beds (OPH),HMIS Participation Rate for Year-Round Beds (OPH),Total Units for Households with Children (OPH),Total Beds for Households with Children (OPH),Total Beds for Households without Children (OPH),Total Beds for Households with only Children (OPH),Year,Region,Population
0,AK,1814,1382,1031,0.5684,1189,625,0,186,646,1117,51,1189,831,650,0.5467,20,122,117,401,766,22,625,551,381,0.6096,69,245,351,29,0,0,0,0%,0,0,0,0,158,158,158,1.0,30,123,35,0,715,715,715,1.0000,55,167,548,0,0,0,0,0%,0,0,0,0,2014,West,"736,283"
1,AL,3724,3212,1985,0.5330,1838,1852,34,477,1394,2294,36,1838,1527,965,0.5250,10,430,202,540,1276,22,1852,1651,986,0.5324,275,854,984,14,34,34,34,1.0,0,0,34,0,53,53,43,0.8113,11,39,14,0,2891,2891,1775,0.6140,417,1206,1685,0,0,0,0,0%,0,0,0,0,2014,Southeast,"4,841,799"
2,AR,2151,1687,944,0.4389,1466,685,0,179,690,1428,33,1466,1114,559,0.3813,0,308,103,449,1002,15,685,573,385,0.5620,76,241,426,18,0,0,0,0%,0,0,0,0,157,157,157,1.0,37,93,64,0,923,923,613,0.6641,122,331,592,0,22,16,16,0.7273,2,9,13,0,2014,Southeast,"2,967,392"
3,AZ,8873,7411,6063,0.6833,4481,4352,40,1418,4845,3983,45,4481,3305,2769,0.6179,229,434,636,2222,2231,28,4352,4066,3254,0.7477,782,2623,1712,17,40,40,40,1.0,0,0,40,0,441,345,344,0.78,117,339,102,0,7358,7358,5568,0.7567,836,2794,4564,0,707,707,47,0.0665,99,292,415,0,2014,Southwest,"6,730,413"
4,CA,44035,38960,31075,0.7057,17869,25959,207,5831,19739,23854,442,17869,15351,11796,0.6601,5300,1450,1827,6424,11065,380,25959,23402,19072,0.7347,4004,13315,12582,62,207,207,207,1.0,0,0,207,0,3504,3432,2938,0.8385,1097,3068,430,6,45580,45250,32954,0.7230,4942,14412,31167,1,2516,2516,1763,0.7007,684,2110,406,0,2014,West,"38,596,972"


In [233]:
s = HIC_Counts.select_dtypes('number').columns

HIC_Counts_Per_Capita = pd.DataFrame(columns=s)

HIC_Counts['Population'] = HIC_Counts['Population'].str.replace(',', '').astype(float)

HIC_Counts_Per_Capita[s] = HIC_Counts[s].div(HIC_Counts['Population'].values, axis=0)

In [234]:
HIC_Counts_Per_Capita.head()

,"Total Year-Round Beds (ES, TH, SH)","Total Non-DV Year-Round Beds (ES, TH, SH)","Total HMIS Year-Round Beds (ES, TH, SH)","HMIS Participation Rate for Year-Round Beds (ES, TH, SH)",Total Year-Round Beds (ES),Total Year-Round Beds (TH),Total Year-Round Beds (SH),"Total Units for Households with Children (ES, TH, SH)","Total Beds for Households with Children (ES, TH, SH)","Total Beds for Households without Children (ES, TH, SH)","Total Beds for Households with only Children (ES, TH, SH)",Total Year-Round Beds (ES).1,Total Non-DV Year-Round Beds (ES),Total HMIS Year-Round Beds (ES),HMIS Participation Rate for Year-Round Beds (ES),Total Seasonal Beds (ES),Total Overflow Beds (ES),Total Units for Households with Children (ES),Total Beds for Households with Children (ES),Total Beds for Households without Children (ES),Total Beds for Households with only Children (ES),Total Year-Round Beds (TH).1,Total Non-DV Year-Round Beds (TH),Total HMIS Year-Round Beds (TH),HMIS Participation Rate for Year-Round Beds (TH),Total Units for Households with Children (TH),Total Beds for Households with Children (TH),Total Beds for Households without Children (TH),Total Beds for Households with only Children (TH),Total Year-Round Beds (SH).1,Total Non-DV Year-Round Beds (SH),Total HMIS Year-Round Beds (SH),Total Units for Households with Children (SH),Total Beds for Households with Children (SH),Total Beds for Households without Children (SH),Total Beds for Households with only Children (SH),Total Year-Round Beds (RRH),Total Non-DV Year-Round Beds (RRH),Total HMIS Year-Round Beds (RRH),Total Units for Households with Children (RRH),Total Beds for Households with Children (RRH),Total Beds for Households without Children (RRH),Total Beds for Households with only Children (RRH),Total Year-Round Beds (PSH),Total Non-DV Year-Round Beds (PSH),Total HMIS Year-Round Beds (PSH),HMIS Participation Rate for Year-Round Beds (PSH),Total Units for Households with Children (PSH),Total Beds for Households with Children (PSH),Total Beds for Households without Children (PSH),Total Beds for Households with only Children (PSH),Total Year-Round Beds (OPH),Total Non-DV Year-Round Beds (OPH),Total HMIS Year-Round Beds (OPH),Total Units for Households with Children (OPH),Total Beds for Households with Children (OPH),Total Beds for Households without Children (OPH),Total Beds for Households with only Children (OPH)
0,0.002464,0.001877,0.001400,7.719858e-07,0.001615,0.000849,0.000000,0.000253,0.000877,0.001517,0.000069,0.001615,0.001129,0.000883,7.425134e-07,0.000027,0.000166,0.000159,0.000545,0.001040,0.000030,0.000849,0.000748,0.000517,8.279425e-07,0.000094,0.000333,0.000477,0.000039,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000215,0.000215,0.000215,0.000041,0.000167,0.000048,0.000000e+00,0.000971,0.000971,0.000971,1.358173e-06,0.000075,0.000227,0.000744,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.0
1,0.000769,0.000663,0.000410,1.100830e-07,0.000380,0.000383,0.000007,0.000099,0.000288,0.000474,0.000007,0.000380,0.000315,0.000199,1.084308e-07,0.000002,0.000089,0.000042,0.000112,0.000264,0.000005,0.000383,0.000341,0.000204,1.099591e-07,0.000057,0.000176,0.000203,0.000003,0.000007,0.000007,0.000007,0.0,0.0,0.000007,0.0,0.000011,0.000011,0.000009,0.000002,0.000008,0.000003,0.000000e+00,0.000597,0.000597,0.000367,1.268124e-07,0.000086,0.000249,0.000348,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.0
2,0.000725,0.000569,0.000318,1.479077e-07,0.000494,0.000231,0.000000,0.000060,0.000233,0.000481,0.000011,0.000494,0.000375,0.000188,1.284967e-07,0.000000,0.000104,0.000035,0.000151,0.000338,0.000005,0.000231,0.000193,0.000130,1.893919e-07,0.000026,0.000081,0.000144,0.000006,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000053,0.000053,0.000053,0.000012,0.000031,0.000022,0.000000e+00,0.000311,0.000311,0.000207,2.237992e-07,0.000041,0.000112,0.000200,0.000000e+00,0.000007,0.000005,0.000005,6.739925e-07,0.000003,0.00000

## Exploratory Data Analysis

In [235]:
HIC_Counts_Per_Capita_stats = HIC_Counts_Per_Capita.describe()
HIC_Counts_Per_Capita_stats

,"Total Year-Round Beds (ES, TH, SH)","Total Non-DV Year-Round Beds (ES, TH, SH)","Total HMIS Year-Round Beds (ES, TH, SH)","HMIS Participation Rate for Year-Round Beds (ES, TH, SH)",Total Year-Round Beds (ES),Total Year-Round Beds (TH),Total Year-Round Beds (SH),"Total Units for Households with Children (ES, TH, SH)","Total Beds for Households with Children (ES, TH, SH)","Total Beds for Households without Children (ES, TH, SH)","Total Beds for Households with only Children (ES, TH, SH)",Total Year-Round Beds (ES).1,Total Non-DV Year-Round Beds (ES),Total HMIS Year-Round Beds (ES),HMIS Participation Rate for Year-Round Beds (ES),Total Seasonal Beds (ES),Total Overflow Beds (ES),Total Units for Households with Children (ES),Total Beds for Households with Children (ES),Total Beds for Households without Children (ES),Total Beds for Households with only Children (ES),Total Year-Round Beds (TH).1,Total Non-DV Year-Round Beds (TH),Total HMIS Year-Round Beds (TH),HMIS Participation Rate for Year-Round Beds (TH),Total Units for Households with Children (TH),Total Beds for Households with Children (TH),Total Beds for Households without Children (TH),Total Beds for Households with only Children (TH),Total Year-Round Beds (SH).1,Total Non-DV Year-Round Beds (SH),Total HMIS Year-Round Beds (SH),Total Units for Households with Children (SH),Total Beds for Households with Children (SH),Total Beds for Households without Children (SH),Total Beds for Households with only Children (SH),Total Year-Round Beds (RRH),Total Non-DV Year-Round Beds (RRH),Total HMIS Year-Round Beds (RRH),Total Units for Households with Children (RRH),Total Beds for Households with Children (RRH),Total Beds for Households without Children (RRH),Total Beds for Households with only Children (RRH),Total Year-Round Beds (PSH),Total Non-DV Year-Round Beds (PSH),Total HMIS Year-Round Beds (PSH),HMIS Participation Rate for Year-Round Beds (PSH),Total Units for Households with Children (PSH),Total Beds for Households with Children (PSH),Total Beds for Households without Children (PSH),Total Beds for Households with only Children (PSH),Total Year-Round Beds (OPH),Total Non-DV Year-Round Beds (OPH),Total HMIS Year-Round Beds (OPH),Total Units for Households with Children (OPH),Total Beds for Households with Children (OPH),Total Beds for Households without Children (OPH),Total Beds for Households with only Children (OPH)
count,459.000000,459.000000,459.000000,4.590000e+02,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,4.590000e+02,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,4.590000e+02,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,4.590000e+02,4.590000e+02,459.000000,4.590000e+02,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,4.590000e+02,459.000000,459.000000,459.000000,4.590000e+02,459.000000,459.000000,459.000000,4.590000e+02,459.000000,459.000000,4.590000e+02,459.000000,459.000000,459.000000,4.590000e+02
mean,0.001368,0.001183,0.000970,2.726265e-07,0.000933,0.000430,0.000006,0.000204,0.000645,0.000709,0.000014,0.000933,0.000797,0.000660,2.639511e-07,0.000082,0.000090,0.000130,0.000425,0.000498,0.000010,0.000430,0.000380,0.000304,2.791410e-07,0.000074,0.000221,0.000205,0.000004,0.000006,0.000006,0.000006,2.485847e-09,5.189379e-09,0.000006,1.042471e-09,0.000426,0.000406,0.000385,0.000110,0.000332,0.000094,3.028103e-07,0.001173,0.001165,0.000895,2.910222e-07,0.000155,0.000447,0.000726,5.279138e-07,0.000189,0.000187,1.281912e-04,0.000040,0.000120,0.000070,9.470181e-08
std,0.001427,0.001372,0.001324,3.070581e-07,0.001109,0.000401,0.000007,0.000246,0.000773,0.000714,0.000016,0.001109,0.001092,0.001072,3.086006e-07,0.000140,0.000245,0.000194,0.000628,0.000539,0.000010,0.000401,0.000365,0.000334,3.126388e-07,0.000076,0.000234,0.000198,0.000009,0.000007,0.000007,0.000007,3.810579e-08,8.017453e-08,0.000007,2.233420e-08,0

In [236]:
HIC_Counts_stats = HIC_Counts.describe()
HIC_Counts_stats

,"Total Year-Round Beds (ES, TH, SH)","Total Non-DV Year-Round Beds (ES, TH, SH)","Total HMIS Year-Round Beds (ES, TH, SH)","HMIS Participation Rate for Year-Round Beds (ES, TH, SH)",Total Year-Round Beds (ES),Total Year-Round Beds (TH),Total Year-Round Beds (SH),"Total Units for Households with Children (ES, TH, SH)","Total Beds for Households with Children (ES, TH, SH)","Total Beds for Households without Children (ES, TH, SH)","Total Beds for Households with only Children (ES, TH, SH)",Total Year-Round Beds (ES).1,Total Non-DV Year-Round Beds (ES),Total HMIS Year-Round Beds (ES),HMIS Participation Rate for Year-Round Beds (ES),Total Seasonal Beds (ES),Total Overflow Beds (ES),Total Units for Households with Children (ES),Total Beds for Households with Children (ES),Total Beds for Households without Children (ES),Total Beds for Households with only Children (ES),Total Year-Round Beds (TH).1,Total Non-DV Year-Round Beds (TH),Total HMIS Year-Round Beds (TH),HMIS Participation Rate for Year-Round Beds (TH),Total Units for Households with Children (TH),Total Beds for Households with Children (TH),Total Beds for Households without Children (TH),Total Beds for Households with only Children (TH),Total Year-Round Beds (SH).1,Total Non-DV Year-Round Beds (SH),Total HMIS Year-Round Beds (SH),Total Units for Households with Children (SH),Total Beds for Households with Children (SH),Total Beds for Households without Children (SH),Total Beds for Households with only Children (SH),Total Year-Round Beds (RRH),Total Non-DV Year-Round Beds (RRH),Total HMIS Year-Round Beds (RRH),Total Units for Households with Children (RRH),Total Beds for Households with Children (RRH),Total Beds for Households without Children (RRH),Total Beds for Households with only Children (RRH),Total Year-Round Beds (PSH),Total Non-DV Year-Round Beds (PSH),Total HMIS Year-Round Beds (PSH),HMIS Participation Rate for Year-Round Beds (PSH),Total Units for Households with Children (PSH),Total Beds for Households with Children (PSH),Total Beds for Households without Children (PSH),Total Beds for Households with only Children (PSH),Total Year-Round Beds (OPH),Total Non-DV Year-Round Beds (OPH),Total HMIS Year-Round Beds (OPH),Total Units for Households with Children (OPH),Total Beds for Households with Children (OPH),Total Beds for Households without Children (OPH),Total Beds for Households with only Children (OPH),Population
count,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,4.590000e+02
mean,7925.978214,6978.405229,5740.398693,0.655189,5592.172113,2291.446623,42.359477,1205.339869,3862.936819,3992.019608,71.021786,5592.172113,4905.204793,4099.766885,0.633570,406.413943,411.442266,827.742919,2714.919390,2823.396514,53.856209,2291.446623,2031.664488,1601.525054,0.670389,377.570806,1147.962963,1126.328976,17.154684,42.359477,41.701525,39.106754,0.026144,0.054466,42.294118,0.010893,1946.185185,1824.172113,1736.187364,448.786492,1367.847495,577.080610,1.257081,6890.869281,6837.603486,5159.581699,0.737714,849.080610,2407.806100,4480.973856,2.089325,854.185185,843.488017,553.311547,163.677560,499.810458,353.694989,0.679739,6.402224e+06
std,13439.490045,12649.362802,11075.348538,0.156946,11520.308598,3066.697532,69.226539,2342.112482,7481.108415,6172.588401,89.627124,11520.308598,10972.801779,9924.162275,0.179344,702.546064,718.117312,2133.591959,6845.078501,4914.340645,70.339950,3066.697532,2752.49